### Importing packages and attributes

In [1]:
import scraping_class
import pandas as pd
import requests,os,time
from datetime import datetime
from time import gmtime, strftime

### Running of connector for HTML scraper - Created by Snorre Raslund

In [2]:
def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(0) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring.
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessment
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case).
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type

    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile\n",
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
           
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
  
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
 
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\\n'+';'.join(map(str,row))) # write log.
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\\n'+';'.join(map(str,row))) # write row to log.
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call.
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\\n'+';'.join(map(str,row))) # write row to log file.
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return call_id

logfile="tripadvisor_scraper.txt" # name your log file.
connector = Connector(logfile)

### HTML scraper - 1 step
The below code gets an overview of all the restaurant in the Copenhagen area

In [3]:
url_init = 'https://www.tripadvisor.dk' # TripAdvisor base url
max_restaurant = 2324 # Restaurants stated on he webpage, to be located in Copenhangen area
init_number = 0 
final_urls = []
year = datetime.now().year
month = datetime.now().month
day = datetime.now().day


while init_number <= max_restaurant:
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S') # Creates a timestamp in the format yyyy-mm-dd h:m:s
    
    url_search = '/RestaurantSearch-g189541-o' # first part of url
    url_search2 = '-a_date.' + str(year) + '__2D__' + str(month) + '__2D__' + str(day) + '-a_people.2-a_time.20%3A00%3A00-a_zur.' + str(year) + '__5F__' + str(month) + '__5F__' + str(day) + '-Copenhag.html' # second part of url
    url_search3 = url_init + url_search + str(init_number) + url_search2 # combined dynamic url
    data, call_id = connector.get(url_search3, 'exam_init') # gathering data
    response = data.ok # checks if datastream is possible
    final_urls.append(url_search3)
    if response == True:
        status = 'Ok'

    print('Status:', status , 'Timestamp:', timestamp, 'links processed:', len(final_urls)) # prints response, timestamp and amount of links processed.
    init_number += 30 # incriments the number by 30 to be put into the url on next call."

Status: Ok Timestamp: 2019-08-26 15:50:06 links processed: 1
Status: Ok Timestamp: 2019-08-26 15:50:07 links processed: 2
Status: Ok Timestamp: 2019-08-26 15:50:07 links processed: 3
Status: Ok Timestamp: 2019-08-26 15:50:08 links processed: 4
Status: Ok Timestamp: 2019-08-26 15:50:09 links processed: 5
Status: Ok Timestamp: 2019-08-26 15:50:10 links processed: 6
Status: Ok Timestamp: 2019-08-26 15:50:11 links processed: 7
Status: Ok Timestamp: 2019-08-26 15:50:11 links processed: 8
Status: Ok Timestamp: 2019-08-26 15:50:12 links processed: 9
Status: Ok Timestamp: 2019-08-26 15:50:13 links processed: 10
Status: Ok Timestamp: 2019-08-26 15:50:14 links processed: 11
Status: Ok Timestamp: 2019-08-26 15:50:14 links processed: 12
Status: Ok Timestamp: 2019-08-26 15:50:15 links processed: 13
Status: Ok Timestamp: 2019-08-26 15:50:16 links processed: 14
Status: Ok Timestamp: 2019-08-26 15:50:16 links processed: 15
Status: Ok Timestamp: 2019-08-26 15:50:17 links processed: 16
Status: Ok Timest

In [4]:
print(init_number)
print(len(final_urls))
overview_urls = pd.DataFrame(final_urls)
pd.options.display.max_colwidth = 200
print(overview_urls.head())

2340
78
                                                                                                                                                            0
0    https://www.tripadvisor.dk/RestaurantSearch-g189541-o0-a_date.2019__2D__8__2D__26-a_people.2-a_time.20%3A00%3A00-a_zur.2019__5F__8__5F__26-Copenhag.html
1   https://www.tripadvisor.dk/RestaurantSearch-g189541-o30-a_date.2019__2D__8__2D__26-a_people.2-a_time.20%3A00%3A00-a_zur.2019__5F__8__5F__26-Copenhag.html
2   https://www.tripadvisor.dk/RestaurantSearch-g189541-o60-a_date.2019__2D__8__2D__26-a_people.2-a_time.20%3A00%3A00-a_zur.2019__5F__8__5F__26-Copenhag.html
3   https://www.tripadvisor.dk/RestaurantSearch-g189541-o90-a_date.2019__2D__8__2D__26-a_people.2-a_time.20%3A00%3A00-a_zur.2019__5F__8__5F__26-Copenhag.html
4  https://www.tripadvisor.dk/RestaurantSearch-g189541-o120-a_date.2019__2D__8__2D__26-a_people.2-a_time.20%3A00%3A00-a_zur.2019__5F__8__5F__26-Copenhag.html


In [5]:
overview_urls.columns = ['Init_links']
overview_urls.to_csv('overview_urls.csv', index = None, header = True) # writes data from df to csv file a

### HTML scraper - 2 step
The code below takes the links generated in step 1 as input, and gives us the individual links to all the restaurants in the Copenhagen area.

In [6]:
page_list = []

for url in final_urls: # loops over all overview sites
    response,call_id = connector.get(url,'exam_links')
    link_locations = response.text.split('href="')[1:] # find all links on site 
    link_list = [] 
    
    for i in link_locations:
        if "Restaurant_Review" in i: # finds all links named something with Restaurant review
            link_list.append(i.partition(" ")[0])

        review_list=  []
    for i in link_list:
        if "Restaurant_Review" in i and "#REVIEWS" not in i and "button" not in i: # same links appart from this removes two alternative versions and only stores one.
            review_list.append(i)

            review_list_u = set(review_list) # remove dublicates

    for i in review_list_u:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S') # Creates a timestamp in the format yyyy-mm-dd h:m:s
        page_list.append(url_init+i) # make list of final urls
        if response == True:
            status = 'Ok'
        
        print('Status:', status , 'Timestamp:', timestamp, 'links processed:', len(page_list)) # prints response, timestamp and amount of links processed.      

Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: Ok Timestamp: 2019-08-26 15:51:05 links processed: 78
Status: 

In [7]:
print(len(page_list))
indvidual_urls = pd.DataFrame(page_list)

2320


In [8]:
indvidual_urls.columns = ['Restaurant_links']
indvidual_urls.to_csv('indvidual_urls.csv', index = None, header = True) # writes df to csv file

### HTML scraper - step 3
The below code takes the links for step 2 as input and retreives all the data that we ask for, for the individual restaurants